In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
#! todo: Làm sao để quy các câu trong dataset về chuẩn utf-8?
#! Giải pháp: Mở file theo dạng khác, sau đó copy vào 1 file khác theo chuẩn utf-8.

In [2]:
import transformers

In [15]:
max_length = 128
batch_size = 32
epochs = 2
# labels = [] # vow
#! label này dùng vào đâu trong code?

In [23]:
import sys
sys.path.append("C:/Users/ASUS/OneDrive/Documents/ML_Scientist/NaturalLanguageProcessing/MyminiprojectIn_NLP_at_school/")
train_df = pd.read_csv('train_data_NLP_DucMinh_to_utf.csv')

test_df = pd.read_csv('test_data_NLP_DucMinh_to_utf8.csv')
valid_df = pd.read_csv('dev_data_NLP_DucMinh_to_utf8.csv')


In [24]:
print(train_df['#1 String'])

0       Amrozi accused his brother, whom he called "th...
1       Yucaipa owned Dominick's before selling the ch...
2       They had published an advertisement on the Int...
3       Around 0335 GMT, Tab shares were up 19 cents, ...
4       The stock rose $2.11, or about 11 percent, to ...
                              ...                        
3473    During the flight, engineers misjudged the ext...
3474    The Web site is registered to Parson under his...
3475    The woman, Mary Kathryn Miller, 55, was arrest...
3476    The Hubble Space Telescope's newest picture of...
3477    He is blocking them until the Air Force assign...
Name: #1 String, Length: 3478, dtype: object


In [5]:
print(f"Total train samples : {train_df.shape[0]}")
print(f"Total validation samples: {test_df.shape[0]}")
print(f"Total test samples: {test_df.shape[0]}")

Total train samples : 3478
Total validation samples: 1650
Total test samples: 1650


In [6]:
print(f"Sentence1: {train_df.loc[1, '#1 String']}")
print(f"Sentence2: {train_df.loc[1, '#2 String']}")
print(f"Similarity: {train_df.loc[1, 'Quality']}")

Sentence1: Yucaipa owned Dominick's before selling the chain to Safeway in 1998 for $2.5 billion.
Sentence2: Yucaipa bought Dominick's in 1995 for $693 million and sold it to Safeway for $1.8 billion in 1998.
Similarity: 0


In [7]:
print("Number of missing values")
print(train_df.isnull().sum())
train_df.dropna(axis=0, inplace=True)

Number of missing values
Quality         0
#1 ID           0
#2 ID           0
#1 String       0
#2 String      17
Column1      3458
_1           3478
_2           3478
_3           3478
_4           3478
_5           3478
_6           3478
dtype: int64


In [8]:
class BertSemanticDataGenerator(tf.keras.utils.Sequence):
    """Generates batches of data.

    Args:
        sentence_pairs: Array of premise and hypothesis input sentences.
        labels: Array of labels.
        batch_size: Integer batch size.
        shuffle: boolean, whether to shuffle the data.
        include_targets: boolean, whether to incude the labels.

    Returns:
        Tuples `([input_ids, attention_mask, `token_type_ids], labels)`
        (or just `[input_ids, attention_mask, `token_type_ids]`
         if `include_targets=False`)
    """

    def __init__(
        self,
        sentence_pairs,
        labels,
        batch_size=batch_size,
        shuffle=True,
        include_targets=True,
    ):
        self.sentence_pairs = sentence_pairs
        self.labels = labels
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.include_targets = include_targets
        # Load our BERT Tokenizer to encode the text.
        # We will use base-base-uncased pretrained model.
        self.tokenizer = transformers.BertTokenizer.from_pretrained(
            "bert-base-uncased", do_lower_case=True
        )
        self.indexes = np.arange(len(self.sentence_pairs))
        self.on_epoch_end()

    def __len__(self):
        # Denotes the number of batches per epoch.
        return len(self.sentence_pairs) // self.batch_size

    def __getitem__(self, idx):
        # Retrieves the batch of index.
        indexes = self.indexes[idx * self.batch_size : (idx + 1) * self.batch_size]
        sentence_pairs = self.sentence_pairs[indexes]

        # With BERT tokenizer's batch_encode_plus batch of both the sentences are
        # encoded together and separated by [SEP] token.
        encoded = self.tokenizer.batch_encode_plus(
            sentence_pairs.tolist(),
            add_special_tokens=True,
            max_length=max_length,
            return_attention_mask=True,
            return_token_type_ids=True,
            pad_to_max_length=True,
            return_tensors="tf",
        )

        # Convert batch of encoded features to numpy array.
        input_ids = np.array(encoded["input_ids"], dtype="int32")
        attention_masks = np.array(encoded["attention_mask"], dtype="int32")
        token_type_ids = np.array(encoded["token_type_ids"], dtype="int32")

        # Set to true if data generator is used for training/validation.
        if self.include_targets:
            labels = np.array(self.labels[indexes], dtype="int32")
            return [input_ids, attention_masks, token_type_ids], labels
        else:
            return [input_ids, attention_masks, token_type_ids]

    def on_epoch_end(self):
        # Shuffle indexes after each epoch if shuffle is set to True.
        if self.shuffle:
            np.random.RandomState(42).shuffle(self.indexes)

In [11]:
train_df["label"] = train_df["Quality"].apply(
    lambda x: 0 if x == 0 else 1 
)
y_train = tf.keras.utils.to_categorical(train_df.label, num_classes=3)

valid_df["label"] = valid_df["Quality"].apply(
    lambda x: 0 if x == 0 else 1
)
y_val = tf.keras.utils.to_categorical(valid_df.label, num_classes=3)

test_df["label"] = test_df["Quality"].apply(
    lambda x: 0 if x == 0 else 1
)
y_test = tf.keras.utils.to_categorical(test_df.label, num_classes=3)

In [10]:
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    # Encoded token ids from BERT tokenizer.
    input_ids = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="input_ids"
    )
    # Attention masks indicates to the model which tokens should be attended to.
    attention_masks = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="attention_masks"
    )
    # Token type ids are binary masks identifying different sequences in the model.
    token_type_ids = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="token_type_ids"
    )
    # Loading pretrained BERT model.
    bert_model = transformers.TFBertModel.from_pretrained("bert-base-uncased")
    # Freeze the BERT model to reuse the pretrained features without modifying them.
    bert_model.trainable = False

    bert_output = bert_model.bert(
        input_ids, attention_mask=attention_masks, token_type_ids=token_type_ids
    )
    sequence_output = bert_output.last_hidden_state
    pooled_output = bert_output.pooler_output
    # Add trainable layers on top of frozen layers to adapt the pretrained features on the new data.
    bi_lstm = tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(64, return_sequences=True)
    )(sequence_output)
    # Applying hybrid pooling approach to bi_lstm sequence output.
    avg_pool = tf.keras.layers.GlobalAveragePooling1D()(bi_lstm)
    max_pool = tf.keras.layers.GlobalMaxPooling1D()(bi_lstm)
    concat = tf.keras.layers.concatenate([avg_pool, max_pool])
    dropout = tf.keras.layers.Dropout(0.3)(concat)
    output = tf.keras.layers.Dense(3, activation="softmax")(dropout)
    model = tf.keras.models.Model(
        inputs=[input_ids, attention_masks, token_type_ids], outputs=output
    )

    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss="categorical_crossentropy",
        metrics=["acc"],
    )


print(f"Strategy: {strategy}")
model.summary()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Strategy: <tensorflow.python.distribute.mirrored_strategy.MirroredStrategy object at 0x00000236F13FEE50>
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, 128)]                0         []                            
                                                                                                  
 attention_masks (InputLaye  [(None, 128)]                0         []                            
 r)                                                                                               
                                                                                                  
 token_type_ids (InputLayer  [(None, 128)]                0         []                            
 )                                                                                      

In [20]:
train_data = BertSemanticDataGenerator(
    train_df[['#1 String', '#2 String']].values.astype("str"),
    y_train,
    batch_size=batch_size,
    shuffle=True,
)
valid_data = BertSemanticDataGenerator(
    valid_df[['#1 String', '#2 String']].values.astype("str"),
    y_val,
    batch_size=batch_size,
    shuffle=False,
)
print(train_df[['#1 String', '#2 String']])

Empty DataFrame
Columns: [#1 String, #2 String]
Index: []


In [ ]:
history = model.fit(
    train_data,
    validation_data=valid_data,
    epochs=epochs,
    use_multiprocessing=True,
    workers=-1,
)